In [ ]:
# import pandas as pd
# import numpy as np
# from sklearn.preprocessing import LabelEncoder
# from sklearn.impute import SimpleImputer
# from sklearn.preprocessing import StandardScaler
# from imblearn.over_sampling import SMOTE

# # Step 1: Load the data
# df = pd.read_csv('radiomics_cleaned_merged.csv')  # Replace with your actual merged filename

# # Check initial memory usage
# print(f"Initial memory usage:\n{df.memory_usage(deep=True)}\n")

# # Step 2: Drop unnecessary columns to reduce memory usage
# df = df.drop(columns=[
#     "PatientID", "BraTS21 ID", "BraTS21 Segmentation Cohort", "BraTS21 MGMT Cohort",
# ], errors="ignore")

# # Step 3: Downcast numerical columns to save memory
# for col in df.select_dtypes(include=["float64"]).columns:
#     df[col] = pd.to_numeric(df[col], downcast="float")

# # Check memory usage again after dropping columns and downcasting
# print(f"Memory usage after optimization:\n{df.memory_usage(deep=True)}\n")

# # Step 4: Keep only patients who are dead (status = 1) and valid OS values
# df = df[df["1-dead 0-alive"] == 1]
# df = df[df["OS"].notna()]
# df["OS"] = pd.to_numeric(df["OS"], errors="coerce")
# df = df[df["OS"] > 0]

# # Step 5: Perform quartile binning on OS
# percentiles = np.percentile(df["OS"], [25, 50, 75])
# bins = [0, percentiles[0], percentiles[1], percentiles[2], np.inf]
# labels = [0, 1, 2, 3]
# df["Survival_Category"] = pd.cut(df["OS"], bins=bins, labels=labels)

# # Step 6: Separate features (X) and target (y)
# X = df.drop(columns=["OS", "Survival_Category", "1-dead 0-alive"], errors="ignore")
# y = df["Survival_Category"]

# # Step 7: Encode categorical variables
# categorical_cols = X.select_dtypes(include=["object"]).columns
# label_encoders = {}
# for col in categorical_cols:
#     le = LabelEncoder()
#     X[col] = le.fit_transform(X[col].astype(str))
#     label_encoders[col] = le

# # Step 8: Impute missing values using median strategy
# imputer = SimpleImputer(strategy="median")
# X_imputed = imputer.fit_transform(X)

# # Step 9: Standardize the features
# scaler = StandardScaler()
# X_scaled = scaler.fit_transform(X_imputed)

# # Step 10: Check the shape of X and y
# print(f"Features shape: {X_scaled.shape}")
# print(f"Target shape: {y.shape}")

# # Step 11: Apply SMOTE to handle class imbalance
# smote = SMOTE(random_state=42)
# X_smote, y_smote = smote.fit_resample(X_scaled, y)

# # Check the number of samples before and after SMOTE
# print(f"Before SMOTE: {X_scaled.shape[0]} samples")
# print(f"After SMOTE: {X_smote.shape[0]} samples")
# print(f"Class distribution after SMOTE:\n{pd.Series(y_smote).value_counts()}")


Initial memory usage:
Index                                       132
PatientID                                  4788
T1c_exponential_firstorder_10Percentile     608
T1c_exponential_firstorder_90Percentile     608
T1c_exponential_firstorder_Energy           608
                                           ... 
EOR                                        3970
Biopsy prior to imaging                    3885
BraTS21 ID                                 4096
BraTS21 Segmentation Cohort                3736
BraTS21 MGMT Cohort                        3686
Length: 20570, dtype: int64

Memory usage after optimization:
Index                                       132
T1c_exponential_firstorder_10Percentile     304
T1c_exponential_firstorder_90Percentile     304
T1c_exponential_firstorder_Energy           608
T1c_exponential_firstorder_Entropy          304
                                           ... 
IDH                                        4361
1-dead 0-alive                              608
OS  

In [ ]:
# from sklearn.model_selection import train_test_split
# from imblearn.over_sampling import SMOTE
# from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
# from sklearn.linear_model import LogisticRegression
# from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
# import xgboost as xgb
# from sklearn.metrics import classification_report

# # Splitting the data into training and testing sets
# X_train, X_test, y_train, y_test = train_test_split(
#     X_scaled, y, test_size=0.1, stratify=y, random_state=42
# )

# # SMOTE to handle class imbalance
# smote = SMOTE(random_state=42)
# X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# # Base models
# rf = RandomForestClassifier(n_estimators=150, random_state=42)
# xgb_clf = xgb.XGBClassifier(n_estimators=200, learning_rate=0.05, max_depth=6, random_state=42, use_label_encoder=False, eval_metric='mlogloss')
# logreg = LogisticRegression(max_iter=1000)
# lda_clf = LinearDiscriminantAnalysis()

# # Voting and stacking classifiers
# voting = VotingClassifier(estimators=[
#     ("rf", rf), ("xgb", xgb_clf), ("logreg", logreg), ("lda", lda_clf)
# ], voting="hard")

# stacking = StackingClassifier(
#     estimators=[("rf", rf), ("xgb", xgb_clf), ("logreg", logreg), ("lda", lda_clf)],
#     final_estimator=RandomForestClassifier()
# )

# # Training the classifiers
# voting.fit(X_train_resampled, y_train_resampled)
# stacking.fit(X_train_resampled, y_train_resampled)

# # Predicting on the test set
# y_pred_voting = voting.predict(X_test)
# y_pred_stacking = stacking.predict(X_test)

# # Classification report for Voting Classifier
# print("\nVoting Classifier Report:")
# print(classification_report(y_test, y_pred_voting))

# # Classification report for Stacking Classifier
# print("\nStacking Classifier Report:")
# print(classification_report(y_test, y_pred_stacking))


C:\Users\Devyansh\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [12:34:40] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\Devyansh\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [12:35:01] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\Devyansh\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [12:35:24] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\Devyansh\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [12:35:37] WARNING: C:\actions-runner\_work\xgboost\xgboost\


Voting Classifier Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       0.00      0.00      0.00         1
           2       0.00      0.00      0.00         1
           3       0.50      1.00      0.67         1

    accuracy                           0.60         5
   macro avg       0.38      0.50      0.42         5
weighted avg       0.50      0.60      0.53         5


Stacking Classifier Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       0.00      0.00      0.00         1
           2       0.33      1.00      0.50         1
           3       0.00      0.00      0.00         1

    accuracy                           0.60         5
   macro avg       0.33      0.50      0.38         5
weighted avg       0.47      0.60      0.50         5



C:\Users\Devyansh\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Devyansh\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Devyansh\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita

In [ ]:
# print(f"Shape of X_train_resampled: {X_train_resampled.shape}")
# print(f"Shape of y_train_resampled: {y_train_resampled.shape}")

Shape of X_train_resampled: (44, 20563)
Shape of y_train_resampled: (44,)


In [ ]:
# # Checking the class distribution after SMOTE
# print(f"Class distribution after SMOTE:\n{y_train_resampled.value_counts()}")

Class distribution after SMOTE:
Survival_Category
0    11
1    11
2    11
3    11
Name: count, dtype: int64


---

In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE

# Step 1: Load the data
df = pd.read_csv('radiomics_cleaned_merged.csv')  # Replace with your actual merged filename

# Check available columns
print(df.columns)

# Step 5: Create the Survival_Category column if it doesn't exist
if 'Survival_Category' not in df.columns:
    percentiles = np.percentile(df["OS"], [25, 50, 75])
    bins = [0, percentiles[0], percentiles[1], percentiles[2], np.inf]
    labels = [0, 1, 2, 3]
    df["Survival_Category"] = pd.cut(df["OS"], bins=bins, labels=labels)

# Step 6: Separate features (X) and target (y)
X = df.drop(columns=["OS", "1-dead 0-alive"], errors="ignore")
y = df["Survival_Category"]

# Step 7: Encode categorical variables
categorical_cols = X.select_dtypes(include=["object"]).columns
label_encoders = {}
for col in categorical_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col].astype(str))
    label_encoders[col] = le

# Step 8: Impute missing values using median strategy
imputer = SimpleImputer(strategy="median")
X_imputed = imputer.fit_transform(X)

# Step 9: Standardize the features
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_imputed)

# Step 10: Check the shape of X and y
print(f"Features shape: {X_scaled.shape}")
print(f"Target shape: {y.shape}")

# Check the class distribution before SMOTE
print(f"Class distribution before SMOTE:\n{y.value_counts()}")

# Step 11: Apply SMOTE to handle class imbalance
smote = SMOTE(sampling_strategy='auto', random_state=42)
X_smote, y_smote = smote.fit_resample(X_scaled, y)

# Step 12: Check the number of samples before and after SMOTE
print(f"Before SMOTE: {X_scaled.shape[0]} samples")
print(f"After SMOTE: {X_smote.shape[0]} samples")

# Check the class distribution after SMOTE
print(f"Class distribution after SMOTE:\n{pd.Series(y_smote).value_counts()}")

Index(['PatientID', 'T1c_exponential_firstorder_10Percentile',
       'T1c_exponential_firstorder_90Percentile',
       'T1c_exponential_firstorder_Energy',
       'T1c_exponential_firstorder_Entropy',
       'T1c_exponential_firstorder_InterquartileRange',
       'T1c_exponential_firstorder_Kurtosis',
       'T1c_exponential_firstorder_Maximum',
       'T1c_exponential_firstorder_MeanAbsoluteDeviation',
       'T1c_exponential_firstorder_Mean',
       ...
       'MGMT index', '1p/19q', 'IDH', '1-dead 0-alive', 'OS', 'EOR',
       'Biopsy prior to imaging', 'BraTS21 ID', 'BraTS21 Segmentation Cohort',
       'BraTS21 MGMT Cohort'],
      dtype='object', length=20569)
Features shape: (76, 20568)
Target shape: (76,)
Class distribution before SMOTE:
Survival_Category
0    19
1    19
2    19
3    19
Name: count, dtype: int64
Before SMOTE: 76 samples
After SMOTE: 76 samples
Class distribution after SMOTE:
Survival_Category
0    19
1    19
2    19
3    19
Name: count, dtype: int64


In [7]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import xgboost as xgb
from sklearn.metrics import classification_report

# Split the already balanced SMOTE data
X_train, X_test, y_train, y_test = train_test_split(
    X_smote, y_smote, test_size=0.1, stratify=y_smote, random_state=42
)

# Define lighter base models to avoid memory issues
rf = RandomForestClassifier(n_estimators=100, max_depth=8, random_state=42)
xgb_clf = xgb.XGBClassifier(
    n_estimators=50,          # reduce number of trees
    max_depth=3,              # smaller trees
    learning_rate=0.1,        # conservative learning
    subsample=0.8,            # use part of data per tree
    colsample_bytree=0.8,     # use part of features
    random_state=42,
    use_label_encoder=False,
    eval_metric='mlogloss'
)
logreg = LogisticRegression(max_iter=500)
lda_clf = LinearDiscriminantAnalysis()

# Ensemble models
voting = VotingClassifier(estimators=[
    ("rf", rf), ("xgb", xgb_clf), ("logreg", logreg), ("lda", lda_clf)
], voting="hard")

stacking = StackingClassifier(
    estimators=[("rf", rf), ("xgb", xgb_clf), ("logreg", logreg), ("lda", lda_clf)],
    final_estimator=LogisticRegression(max_iter=500)  # simpler meta-model
)

# Fit models
voting.fit(X_train, y_train)
stacking.fit(X_train, y_train)

# Predictions
y_pred_voting = voting.predict(X_test)
y_pred_stacking = stacking.predict(X_test)

# Reports
print("\nVoting Classifier Report:")
print(classification_report(y_test, y_pred_voting))

print("\nStacking Classifier Report:")
print(classification_report(y_test, y_pred_stacking))

C:\Users\Devyansh\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [16:15:46] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\Devyansh\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [16:15:54] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\Devyansh\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [16:16:03] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
C:\Users\Devyansh\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [16:16:09] WARNING: C:\actions-runner\_work\xgboost\xgboost\


Voting Classifier Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       0.67      1.00      0.80         2
           2       0.50      0.50      0.50         2
           3       1.00      0.50      0.67         2

    accuracy                           0.75         8
   macro avg       0.79      0.75      0.74         8
weighted avg       0.79      0.75      0.74         8


Stacking Classifier Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         2

    accuracy                           1.00         8
   macro avg       1.00      1.00      1.00         8
weighted avg       1.00      1.00      1.00         8



---

In [8]:
# Train Random Forest
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)
print("\nRandom Forest Report:")
print(classification_report(y_test, y_pred_rf))


Random Forest Report:
              precision    recall  f1-score   support

           0       1.00      0.50      0.67         2
           1       0.33      0.50      0.40         2
           2       0.00      0.00      0.00         2
           3       0.50      0.50      0.50         2

    accuracy                           0.38         8
   macro avg       0.46      0.38      0.39         8
weighted avg       0.46      0.38      0.39         8



In [9]:
# Train XGBoost
xgb_clf.fit(X_train, y_train)
y_pred_xgb = xgb_clf.predict(X_test)
print("\nXGBoost Report:")
print(classification_report(y_test, y_pred_xgb))

C:\Users\Devyansh\AppData\Roaming\Python\Python312\site-packages\xgboost\training.py:183: UserWarning: [16:16:36] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)



XGBoost Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         2

    accuracy                           1.00         8
   macro avg       1.00      1.00      1.00         8
weighted avg       1.00      1.00      1.00         8



In [10]:
# Train Logistic Regression
logreg.fit(X_train, y_train)
y_pred_logreg = logreg.predict(X_test)
print("\nLogistic Regression Report:")
print(classification_report(y_test, y_pred_logreg))


Logistic Regression Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         2
           1       0.00      0.00      0.00         2
           2       0.00      0.00      0.00         2
           3       0.17      0.50      0.25         2

    accuracy                           0.12         8
   macro avg       0.04      0.12      0.06         8
weighted avg       0.04      0.12      0.06         8



C:\Users\Devyansh\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Devyansh\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\Devyansh\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capita

In [11]:
# Train LDA
lda_clf.fit(X_train, y_train)
y_pred_lda = lda_clf.predict(X_test)
print("\nLDA Report:")
print(classification_report(y_test, y_pred_lda))


LDA Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00         2
           1       1.00      1.00      1.00         2
           2       1.00      1.00      1.00         2
           3       1.00      1.00      1.00         2

    accuracy                           1.00         8
   macro avg       1.00      1.00      1.00         8
weighted avg       1.00      1.00      1.00         8



---

# Optuna

In [4]:
import optuna
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import xgboost as xgb
from sklearn.model_selection import cross_val_score, StratifiedKFold
import warnings

warnings.filterwarnings("ignore")  # Suppress warnings for cleaner output
cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

In [5]:
def rf_objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 200),
        "max_depth": trial.suggest_int("max_depth", 3, 15),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 10),
        "min_samples_leaf": trial.suggest_int("min_samples_leaf", 1, 5),
        "random_state": 42,
        "n_jobs": -1
    }
    model = RandomForestClassifier(**params)
    return cross_val_score(model, X_smote, y_smote, cv=cv, scoring="accuracy").mean()

rf_study = optuna.create_study(direction="maximize")
rf_study.optimize(rf_objective, n_trials=30)


[I 2025-05-08 13:32:19,284] A new study created in memory with name: no-name-4b434489-5b5b-4294-9eca-a0e91cb45f2c
[I 2025-05-08 13:32:19,589] Trial 0 finished with value: 0.28974358974358977 and parameters: {'n_estimators': 52, 'max_depth': 15, 'min_samples_split': 5, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.28974358974358977.
[I 2025-05-08 13:32:20,211] Trial 1 finished with value: 0.3030769230769231 and parameters: {'n_estimators': 144, 'max_depth': 15, 'min_samples_split': 7, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.3030769230769231.
[I 2025-05-08 13:32:20,524] Trial 2 finished with value: 0.32974358974358975 and parameters: {'n_estimators': 59, 'max_depth': 5, 'min_samples_split': 3, 'min_samples_leaf': 2}. Best is trial 2 with value: 0.32974358974358975.
[I 2025-05-08 13:32:21,045] Trial 3 finished with value: 0.3169230769230769 and parameters: {'n_estimators': 131, 'max_depth': 5, 'min_samples_split': 10, 'min_samples_leaf': 1}. Best is trial 2 with valu

In [6]:
def xgb_objective(trial):
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 30, 150),
        "max_depth": trial.suggest_int("max_depth", 2, 6),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.2),
        "subsample": trial.suggest_float("subsample", 0.6, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.6, 1.0),
        "eval_metric": "mlogloss",
        "use_label_encoder": False,
        "random_state": 42
    }
    model = xgb.XGBClassifier(**params)
    return cross_val_score(model, X_smote, y_smote, cv=cv, scoring="accuracy").mean()

xgb_study = optuna.create_study(direction="maximize")
xgb_study.optimize(xgb_objective, n_trials=30)

[I 2025-05-08 13:32:36,103] A new study created in memory with name: no-name-9b2e03a3-0b80-47e3-83dc-f19bf8dadfd2
[I 2025-05-08 13:32:47,108] Trial 0 finished with value: 0.8425641025641025 and parameters: {'n_estimators': 74, 'max_depth': 2, 'learning_rate': 0.16062846732697408, 'subsample': 0.6731672981908506, 'colsample_bytree': 0.691987854135004}. Best is trial 0 with value: 0.8425641025641025.
[I 2025-05-08 13:33:02,763] Trial 1 finished with value: 0.8174358974358974 and parameters: {'n_estimators': 91, 'max_depth': 6, 'learning_rate': 0.13875722155695103, 'subsample': 0.6949464600938782, 'colsample_bytree': 0.7050992432260849}. Best is trial 0 with value: 0.8425641025641025.
[I 2025-05-08 13:33:12,806] Trial 2 finished with value: 0.9743589743589745 and parameters: {'n_estimators': 75, 'max_depth': 2, 'learning_rate': 0.1951070594609566, 'subsample': 0.8232778367252738, 'colsample_bytree': 0.9420648881812127}. Best is trial 2 with value: 0.9743589743589745.
[I 2025-05-08 13:33:2

In [7]:
def logreg_objective(trial):
    params = {
        "C": trial.suggest_float("C", 0.01, 10.0, log=True),
        "solver": trial.suggest_categorical("solver", ["lbfgs", "liblinear", "saga"]),
        "max_iter": 1000,
        "random_state": 42
    }
    model = LogisticRegression(**params)
    return cross_val_score(model, X_smote, y_smote, cv=cv, scoring="accuracy").mean()

logreg_study = optuna.create_study(direction="maximize")
logreg_study.optimize(logreg_objective, n_trials=20)

[I 2025-05-08 13:39:35,425] A new study created in memory with name: no-name-7d15728b-a0d9-41fa-b670-f2b5685ab1ee
[I 2025-05-08 13:39:37,579] Trial 0 finished with value: 0.27743589743589747 and parameters: {'C': 8.35402817287357, 'solver': 'liblinear'}. Best is trial 0 with value: 0.27743589743589747.
[I 2025-05-08 13:40:21,084] Trial 1 finished with value: 0.2764102564102564 and parameters: {'C': 8.263847423092232, 'solver': 'saga'}. Best is trial 0 with value: 0.27743589743589747.
[I 2025-05-08 13:40:21,440] Trial 2 finished with value: 0.2764102564102564 and parameters: {'C': 3.505023580832271, 'solver': 'lbfgs'}. Best is trial 0 with value: 0.27743589743589747.
[I 2025-05-08 13:40:22,905] Trial 3 finished with value: 0.2907692307692308 and parameters: {'C': 0.05069314455007318, 'solver': 'liblinear'}. Best is trial 3 with value: 0.2907692307692308.
[I 2025-05-08 13:40:23,880] Trial 4 finished with value: 0.2764102564102564 and parameters: {'C': 0.04912300897911432, 'solver': 'lbfg

In [9]:
# def lda_objective(trial):
#     solver = trial.suggest_categorical("solver", ["svd", "lsqr", "eigen"])
#     shrinkage = None
#     if solver in ["lsqr", "eigen"]:
#         shrinkage = trial.suggest_categorical("shrinkage", ["auto", None])

#     model = LinearDiscriminantAnalysis(solver=solver, shrinkage=shrinkage)
#     return cross_val_score(model, X_smote, y_smote, cv=cv, scoring="accuracy").mean()

# lda_study = optuna.create_study(direction="maximize")
# lda_study.optimize(lda_objective, n_trials=15)

In [10]:
# Instantiate best models
best_rf = RandomForestClassifier(**rf_study.best_params, n_jobs=-1)
best_xgb = xgb.XGBClassifier(**xgb_study.best_params)
best_logreg = LogisticRegression(**logreg_study.best_params)